In [1]:
%load_ext sql

In [10]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from faker import Faker

In [12]:
pd.set_option('display.max_rows', 12)
pd.set_option('display.min_rows', 6)

In [3]:
fake = Faker()

base_url = "http://fastapi:8000"
headers = {"Content-Type": "application/json"}

print("customers")
# Create customers
for customer_id in range(1, 11):
    data = {
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email(),
        "joined_at": fake.iso8601()
    }
    response = requests.post(f"{base_url}/customers/", json=data, headers=headers)
    print(response.status_code, response.json())

print("vendors")
# Create vendors
for vendor_id in range(1, 6):
    data = {
        "vendor_name": fake.company(),
        "region": fake.country_code()
    }
    response = requests.post(f"{base_url}/vendors/", json=data, headers=headers)
    print(response.status_code, response.json())

print("inventory per vendor")
# Add inventory per vendor
for vendor_id in range(1, 6):
    for item_id in range(1, 11):
        params = {"vendor_id": vendor_id}
        data = {
            "item_name": fake.word(),
            "category": fake.word(),
            "price": fake.random_int(min=10, max=500),
        }
        response = requests.post(f"{base_url}/inventory/", json=data, headers=headers, params=params)
        print(response.status_code, response.json())

print("cart and checkout twice and add to cart again!")
for customer_id in range(1, 11):
    for _ in range(2):  # Checkout twice
        cart_data = {
            "user_id": customer_id,
            "cart": [
                {
                    "item_id": str(fake.random_int(min=1, max=10)),
                    "vendor_id": str(fake.random_int(min=1, max=5)),
                    "qty": fake.random_int(min=1, max=5)
                }
            ]
        }
        response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
        print(response.status_code, response.json())
        
        response = requests.post(f"{base_url}/checkout/", json={"user_id": customer_id}, headers=headers)
        print(response.status_code, response.text)
    
    # Add to cart one more time
    cart_data = {
        "user_id": customer_id,
        "cart": [
            {
                "item_id": str(fake.random_int(min=1, max=10)),
                "vendor_id": str(fake.random_int(min=1, max=5)),
                "qty": fake.random_int(min=1, max=5)
            }
        ]
    }
    response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
    print(response.status_code, response.json())

customers
201 {'id': 1, 'first_name': 'Jackson', 'last_name': 'Freeman', 'email': 'eric73@example.org', 'joined_at': '2025-03-16T10:31:19.272581'}
201 {'id': 2, 'first_name': 'Bradley', 'last_name': 'Bailey', 'email': 'chandlercharlotte@example.net', 'joined_at': '2025-03-16T10:31:19.283771'}
201 {'id': 3, 'first_name': 'Scott', 'last_name': 'Cook', 'email': 'gcharles@example.net', 'joined_at': '2025-03-16T10:31:19.291722'}
201 {'id': 4, 'first_name': 'Matthew', 'last_name': 'Harding', 'email': 'woodtoni@example.net', 'joined_at': '2025-03-16T10:31:19.300404'}
201 {'id': 5, 'first_name': 'John', 'last_name': 'Tucker', 'email': 'aprilbrown@example.net', 'joined_at': '2025-03-16T10:31:19.309857'}
201 {'id': 6, 'first_name': 'Maria', 'last_name': 'Valdez', 'email': 'hensleydanielle@example.com', 'joined_at': '2025-03-16T10:31:19.317993'}
201 {'id': 7, 'first_name': 'Ashley', 'last_name': 'Carter', 'email': 'alison88@example.com', 'joined_at': '2025-03-16T10:31:19.327727'}
201 {'id': 8, 'f

In [4]:
connection_string = f"postgresql://user:password@db:5432/ecommerce"
engine = create_engine(connection_string)
get_ipython().run_line_magic('sql', connection_string)

Connecting to 'postgresql://user:***@db:5432/ecommerce'

In [5]:
%sql \l

Running query in 'postgresql://user:***@db:5432/ecommerce'

Name,Owner,Encoding,Collate,Ctype,Access privileges
ecommerce,user,UTF8,en_US.utf8,en_US.utf8,None
postgres,user,UTF8,en_US.utf8,en_US.utf8,None
template0,user,UTF8,en_US.utf8,en_US.utf8,=c/useruser=CTc/user
template1,user,UTF8,en_US.utf8,en_US.utf8,=c/useruser=CTc/user


In [6]:
%sql \dt

Running query in 'postgresql://user:***@db:5432/ecommerce'

Schema,Name,Type,Owner
public,customer,table,user
public,inventory,table,user
public,vendor,table,user


In [7]:
table_list = [
    "customer", "inventory", "vendor"
]

In [8]:
for table in table_list:
    print(f"{table}")
    schema_query = f"columns --table {table}"
    display(get_ipython().run_line_magic('sqlcmd', schema_query))

customer


name,type,nullable,default,autoincrement,comment
id,INTEGER,False,nextval('customer_id_seq'::regclass),True,None
first_name,TEXT,False,None,False,None
last_name,TEXT,False,None,False,None
email,TEXT,False,None,False,None
joined_at,TIMESTAMP,False,None,False,None


inventory


name,type,nullable,default,autoincrement,comment
id,INTEGER,False,nextval('inventory_id_seq'::regclass),True,None
item_name,TEXT,False,None,False,None
category,TEXT,False,None,False,None
price,INTEGER,False,None,False,None
updated_at,TIMESTAMP,False,None,False,None
vendor_id,INTEGER,True,None,False,None


vendor


name,type,nullable,default,autoincrement,comment
id,INTEGER,False,nextval('vendor_id_seq'::regclass),True,None
vendor_name,TEXT,False,None,False,None
region,TEXT,False,None,False,None
joined_at,TIMESTAMP,False,None,False,None


In [13]:
for table in table_list:
    print(f"{table}")
    display(pd.read_sql(f"select * from {table} limit 100", engine))

customer


,id,first_name,last_name,email,joined_at
0,1,Jackson,Freeman,eric73@example.org,2025-03-16 10:31:19.272581
1,2,Bradley,Bailey,chandlercharlotte@example.net,2025-03-16 10:31:19.283771
2,3,Scott,Cook,gcharles@example.net,2025-03-16 10:31:19.291722
3,4,Matthew,Harding,woodtoni@example.net,2025-03-16 10:31:19.300404
4,5,John,Tucker,aprilbrown@example.net,2025-03-16 10:31:19.309857
5,6,Maria,Valdez,hensleydanielle@example.com,2025-03-16 10:31:19.317993
6,7,Ashley,Carter,alison88@example.com,2025-03-16 10:31:19.327727
7,8,Allison,Taylor,prichardson@example.com,2025-03-16 10:31:19.338297
8,9,Michelle,Spencer,ramirezmark@example.com,2025-03-16 10:31:19.349096
9,10,Sheryl,Hanson,daniellehall@example.com,2025-03-16 10:31:19.359303


inventory


,id,item_name,category,price,updated_at,vendor_id
0,1,how,but,469,2025-03-16 10:31:19.416024,1
1,2,board,production,68,2025-03-16 10:31:19.426401,1
2,3,risk,any,458,2025-03-16 10:31:19.437527,1
...,...,...,...,...,...,...
47,48,article,both,299,2025-03-16 10:31:19.880480,5
48,49,hundred,development,189,2025-03-16 10:31:19.890761,5
49,50,treat,agent,162,2025-03-16 10:31:19.901447,5


vendor


,id,vendor_name,region,joined_at
0,1,Weaver Inc,RO,2025-03-16 10:31:19.370019
1,2,Burton-Morrison,AD,2025-03-16 10:31:19.378211
2,3,"Harding, Johnson and Rogers",BW,2025-03-16 10:31:19.388256
3,4,Crawford Ltd,KH,2025-03-16 10:31:19.397292
4,5,"Wilson, Harper and Young",MK,2025-03-16 10:31:19.406434
